In [1]:
from abc import ABC, abstractmethod

class EstadoProblema:
    """
    La clase EstadoProblema es abstracta.
    Representa un estado o configuración del problema a resolver.
    
    Es una interfaz simplificada para utilizarse
    en los algoritmos de búsqueda del curso.
    
    Al definir un problema particular hay que implementar los métodos
    abstractos
    """
    
    @abstractmethod
    def expand():
        """
        :return: el conjunto de estados sucesores
        """
        pass
    
    @abstractmethod
    def get_depth():
        """
        :return: la profundidad del estado
        """
        pass
        
    @abstractmethod
    def get_parent():
        """
        :return: referencia al estado predecesor o padre
        """
        pass


In [2]:
from functools import reduce
import random

# La secuencia del 0 al 15
# 0 representará el espacio en blanco
seq = list(range(0,16))

# Cuatro posibles acciones para nuestro agente
# Mover una ficha en dirección: 
# izquierda (E), derecha (W), arriba (N), o abajo (S)
actions = ['E','W','N','S']

# Representaremos las configuraciones con bits
# Definimos algunas funciones útiles
# Recorre un bloque de 4 bits de unos a la posición i
x_mask = lambda i: 15<<(4*i)

# Extrae los cuatro bits que están en la posción i
# en la configuración c
# El rompecabezas tiene 16 posiciones (16X4 = 64 bits)
extract = lambda i,c: (c&(x_mask(i)))>>(4*i)

# Verifica si la posición z es la última columna
e_most = lambda z: (z%4)==3

# Verifica si la posición z es la primera columna
w_most = lambda z: (z%4)==0

# Verifica si la posición z es el primer renglón
n_most = lambda z: z<=3

# Verifica si la posición z es el último renglón
s_most = lambda z:z>=12

# Establecemos un diccionario con las acciones posibles
# para cada posición del rompecabezas
valid_moves = {i:list(filter(lambda action:\
((not action=='E') or (not e_most(i))) and \
((not action=='W') or (not w_most(i))) and \
((not action=='S') or (not s_most(i))) and \
((not action=='N') or (not n_most(i))),actions)) for i in seq}

# Realiza el movimiento hacía la izquierda
def move_east(puzzle):
    """
    :param puzzle: el rompecabezas
    """
    if(not e_most(puzzle.zero)):
        puzzle.zero += 1;
        mask = x_mask(puzzle.zero)
        puzzle.configuration = \
        (puzzle.configuration&mask)>>4 | \
        (puzzle.configuration&~mask)

# Realiza el movimiento hacía la derecha
def move_west(puzzle):
    if(not w_most(puzzle.zero)):
        puzzle.zero -= 1;
        mask = x_mask(puzzle.zero)
        puzzle.configuration = \
        (puzzle.configuration&mask)<<4 | \
        (puzzle.configuration&~mask)

# Realiza el movimiento hacía arriba
def move_north(puzzle):
    if(not n_most(puzzle.zero)):
        puzzle.zero -= 4;
        mask = x_mask(puzzle.zero)
        puzzle.configuration = \
        (puzzle.configuration&mask)<<16 | \
        (puzzle.configuration&~mask)

# Realiza el movimiento hacía abajo
def move_south(puzzle):
    if(not s_most(puzzle.zero)):
        puzzle.zero += 4;
        mask = x_mask(puzzle.zero)
        puzzle.configuration = \
        (puzzle.configuration&mask)>>16 | \
        (puzzle.configuration&~mask)

class Puzzle(EstadoProblema):
    """
    Rompecabezas del 15
    """
    
    
    def __init__(self, parent=None, action =None, depth=0):
        """
        Puede crearse un rompecabezas ordenado al no especificar
        parámetros del constructor.
        También se puede crear una nueva configuración a 
        partir de una configuración dada en parent.
        :param parent: configuración de referencia.
        :param action: la acción que se aplica a parent para
        generar la configuración sucesora.
        :depth la profundidad del estado a crear
        """
        self.parent = parent
        self.depth = depth
        if(parent == None):
            self.configuration =  \
                reduce(lambda x,y: x | (y << 4*(y-1)), seq)
            # posición del cero
            self.zero = 15
        else:
            self.configuration = parent.configuration
            self.zero = parent.zero
            if(action != None):
                self.move(action)

    def __str__(self):
        """
        :return: un string que representa 
        la configuración del rompecabezas
        """
        return '\n'+''.join(list(map(lambda i:\
        format(extract(i,self.configuration)," x")+\
        ('\n' if (i+1)%4==0 else ''),seq)))+'\n'

    def __repr__(self):
        """
        :return: representación texto de la configuración
        """
        return self.__str__()

    def __eq__(self,other):
        """
        :param other: la otra configuración con la que se comparará
        el objeto
        :return: verdadero cuando el objeto y el parámetro
        tienen la misma configuración.
        """
        return (isinstance(other, self.__class__)) and \
        (self.configuration==other.configuration)

    def __ne__(self,other):
        """
        :param other: la otra configuración con la que se comparará
        el objeto
        :return: verdadero cuando el objeto y el parámetro
        no tienen la misma configuración
        """
        return not self.__eq__(other)
        
    def __lt__(self,other):
        """
        :param other: la otra configuración con la que se comparará
        el objeto
        :return: verdadero cuando la profundidad del objeto
        es menor que la del argumento
        """
        return self.depth < other.depth

    def __hash__(self):
        """
        :return: un número hash para poder usar la configuración en 
        un diccionario, delegamos al hash de un entero
        """
        return hash(self.configuration)

    def move(self,action):
        """
        Realiza un movimiento de ficha.
        Debemos imaginar que el espacio se mueve en la dirección
        especificada por acción
        :param action: la acción a realizar
        """
        if(action =='E'):
            move_east(self)
        if(action =='W'):
            move_west(self)
        if(action =='N'):
            move_north(self)
        if(action =='S'):
            move_south(self)
        return self


    @staticmethod
    def to_list(puzzle):
        """
        Convertimos la configuración a una lista de números
        :param puzzle: la configuración a convertir
        :return la lista con enteros
        """
        return [extract(i,puzzle.configuration) for i in seq]

    def shuffle(self,n):
        """
        Desordena de manera aleatoria el rompecabezas.
        :param n: el número de movimientos aleatorios a aplicar
        """
        for i in range(0,n):
            self.move(random.choice(valid_moves[self.zero]))
        return self

    def expand(self):
        """
        Los sucesores del estado, quitamos el estado padre
        """
        #filtering the path back to parent
        return list(filter(lambda x: \
        (x!=self.parent), \
        [Puzzle(self,action,self.depth+1) \
        for action in valid_moves[self.zero]]))
    
    def get_depth(self):
        """
        :return: la profundidad del estado
        """
        return self.depth
    
    def get_parent(self):
        """
        :return: el nodo predecesor (padre) del estado 
        """
        return self.parent

In [3]:
# No indicamos un padre, 
# el rompecabezas estará ordenado
# y su profundidad será cero
puzzle = Puzzle()
print("Configuración:\n",puzzle)
print("Profundidad:\n",puzzle.get_depth())
print("Estado predecesor:\n",puzzle.get_parent())

Configuración:
 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e f 0


Profundidad:
 0
Estado predecesor:
 None


In [4]:
puzzle.shuffle(10)
print("Rompecabezas revuelto:\n",puzzle)

Rompecabezas revuelto:
 
 1 2 0 4
 5 6 3 8
 9 a 7 c
 d e b f




In [5]:
from collections import deque

# trajectory nos regresará la trayectoria a partir de un estado
def trajectory(end):
    # nos valemos de un deque para almacenar la ruta
    sequence = deque()
    # agregamos el estado final o meta
    sequence.append(end)
    # nos vamos regresando al estado predecesor mientras este exista
    while end.get_parent():
        # nos movemos al predecesor
        end = end.get_parent()
        # lo agregamos a la lista
        sequence.append(end)
    # invertimos el orden de la secuencia
    sequence.reverse()
    # lo regresamos como una lista
    return list(sequence)


In [6]:
ordenado = Puzzle()
print("ordenado:",ordenado)
print("la profundidad del estado ordenado:",ordenado.get_depth())

ordenado: 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e f 0


la profundidad del estado ordenado: 0


In [7]:
#Obtenemos los sucesores del estado ordenado
sucesores = ordenado.expand() 
# imprimimos los sucesores
print("sucesores del estado ordenado:",sucesores)

sucesores del estado ordenado: [
 1 2 3 4
 5 6 7 8
 9 a b c
 d e 0 f

, 
 1 2 3 4
 5 6 7 8
 9 a b 0
 d e f c

]


In [8]:
# el primer sucesor
primer_sucesor = sucesores[0]
print("el primer sucesor del estado ordenado:",primer_sucesor)
# imprimimos su profundidad
print("su profundidad:",primer_sucesor.get_depth())
# sucesores del primer sucesor del estado ordenado
sucesores_primer_sucesor = primer_sucesor.expand()
# imprimimos los sucesores a profundidad 
print("los sucesores del primer sucesor:",sucesores_primer_sucesor)

el primer sucesor del estado ordenado: 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e 0 f


su profundidad: 1
los sucesores del primer sucesor: [
 1 2 3 4
 5 6 7 8
 9 a b c
 d 0 e f

, 
 1 2 3 4
 5 6 7 8
 9 a 0 c
 d e b f

]


In [9]:
primer_sucesor_primer_sucesor = sucesores_primer_sucesor[0]
print("primer sucesor del primer sucesor:",primer_sucesor_primer_sucesor)

primer sucesor del primer sucesor: 
 1 2 3 4
 5 6 7 8
 9 a b c
 d 0 e f




In [10]:
ruta = trajectory(primer_sucesor_primer_sucesor)
print("ruta desde el estado ordenado: ",ruta)

ruta desde el estado ordenado:  [
 1 2 3 4
 5 6 7 8
 9 a b c
 d e f 0

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e 0 f

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 d 0 e f

]


In [11]:
class DFS:

    @staticmethod    
    def search(start,stop):
        """
        Realiza la búsqueda primero en anchura
        :param start: el estado inicial
        :param stop: una función de paro
        """
        # usamos deque para la agenda que será una cola
        agenda = deque()
        # un conjunto basado en tabla de dispersión para
        # registrar los estados expandidos
        explored = set()
        # verificamos la condición trivial
        if(stop(start)):
            # regresamos la ruta trivial
            return trajectory(start)
        # agregamos el primer estado a la agenda
        agenda.append(start)
        # mientras la agenda tenga elementos
        while(agenda):
            # sacamos el tope
            nodo = agenda.pop()
            # lo agregamos a los expandidos
            explored.add(nodo)
            # para cada sucesor del nodo
            for child in nodo.expand():
                # si el sucesor es la meta 
                if stop(child):
                    # recuperamos la ruta y la regresamos
                    return trajectory(child)
                # si el nodo no se ha previamente expandido
                elif not child in explored:
                    # agregamos los sucesores a la agenda
                    agenda.append(child)
        # en caso de que no haya ruta
        # (instrucción redundante)
        return None

In [12]:
# Un nuevo rompecabezas
puzzle = Puzzle()
# 20 movimeintos aleatorios
puzzle.shuffle(5)
print("rompecabezas desordenado:",puzzle)

rompecabezas desordenado: 
 1 2 3 4
 5 6 7 8
 9 a b 0
 d e f c




In [13]:
# la función de paro evalua a cierto cuando el estado es igual al rompecabezas ordenado
ruta = BFS.search(puzzle,lambda s:s==Puzzle())
# imprimimos la ruta
print(ruta)

[
 1 2 3 4
 5 6 7 8
 9 a b 0
 d e f c

, 
 1 2 3 4
 5 6 7 8
 9 a b c
 d e f 0

]
